# ResNet

**Goal:** Based on this plot, Lukas thought the issue was that for the "fucked up clusterings" we weren't getting a good separation even in latent space.

**Plan:** Lukas thought the issue was that we didn't have a good enough feature extractor for the pixels, so he suggested trying ResNet on the task to have a pretrained backbone :) 

I'll put it together w/ the ISA model, b/c that one didn't take so long to train.

In [1]:
import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mlp
from mpl_toolkits.axes_grid1 import make_axes_locatable

import json, yaml, os
os.sys.path.append('code')

from plotting import plot_kslots, plot_kslots_iters
from data import make_batch
from model import SoftPositionalEmbed, build_grid
from torch.nn import init
from train import hungarian_matching

import torch
import torch.nn.functional as F

%load_ext autoreload
%autoreload 2

In [2]:
from torchvision.models import resnet34

In [18]:
resnet_og = resnet34(pretrained=True)

In [32]:
resnet = torch.nn.Sequential(*(list(resnet_og.children())[:-2]))

In [39]:
len(list(resnet_og.children()))

10

In [33]:
bs = 100

X, Y, mask = make_batch(N_events=bs, N_clusters=2)


In [34]:
X_3chan = X.repeat(1,3,1,1)
X_3chan.shape

torch.Size([100, 3, 32, 32])

In [35]:
out = resnet(X_3chan)

In [36]:
out.shape

torch.Size([100, 512, 1, 1])

In [37]:
resnet

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con